In [1]:
using Pkg
# Activate environment that has ProgressiveHedging installed
# Pkg.activate("/Users/jmaack/SIIP")
home = homedir()
Pkg.activate(joinpath(home,".julia/environments/v1.1"))
Pkg.status()

    Status `~/.julia/environments/v1.1/Project.toml`
  [a93c6f00] DataFrames v0.18.3
  [60bf3e95] GLPK v0.10.0
  [7073ff75] IJulia v1.18.1
  [b6b21f68] Ipopt v0.5.4
  [4076af6c] JuMP v0.19.2
  [da04e1cc] MPI v0.9.0
  [774612a8] ParameterJuMP v0.1.0 [`~/jdev/ParameterJuMP`]
  [070f1c60] ProgressiveHedging v0.1.0 [`~/jdev/ProgressiveHedging`]
  [d330b81b] PyPlot v2.8.1
  [34f15cae] StructJuMP v0.1.0 [`~/jdev/StructJuMP`]
  [9e3dc215] TimeSeries v0.15.0
  [a759f4b9] TimerOutputs v0.5.0


In [2]:
using Distributed
const WORKERS = 2
if nworkers() < WORKERS
    diff = (nprocs() == nworkers() ? WORKERS : WORKERS - nworkers())
    println("Adding $diff worker processes.")
    Distributed.addprocs(diff)
end
# Distributed.addprocs(1)
@everywhere using ProgressiveHedging
const PH = ProgressiveHedging
using Ipopt

using StructJuMP

Adding 2 worker processes.


In [3]:
nscen = 4
nbranch = 2
# Parameters
c = [1.0, 10.0, 0.01]
d = 7.0
a = 16.0

α = 1.0
β = 1.0
γ = 1.0
δ = 1.0
ϵ = 1.0

s1 = 8.0
s2 = 4.0
s11 = 9.0
s12 = 16.0
s21 = 5.0
s22 = 18.0

# First stage
root_model = StructuredModel(num_scenarios=nbranch)
@variable(root_model, x[1:3] >= 0.0)
#@variable(root_model, F >= 0.0)
@objective(root_model, Min, sum(c.*x))
@constraint(root_model, x[3] <= 1.0)

# Second stage
mid_model_1 = StructuredModel(parent=root_model, id=1, num_scenarios=nbranch, prob=0.5)
@variable(mid_model_1, y1 >= 0.0)
@objective(mid_model_1, Min, d*y1)
@constraint(mid_model_1, α*sum(x) + β*y1 >= s1)
mid_model_2 = StructuredModel(parent=root_model, id=2, num_scenarios=nbranch, prob=0.5)
@variable(mid_model_2, y2 >= 0.0)
@objective(mid_model_2, Min, d*y2)
@constraint(mid_model_2, α*sum(x) + β*y2 >= s2)
;

# Third stage
leaf_11 = StructuredModel(parent=mid_model_1, id=11, prob=0.75)
# @variable(leaf_11, z11 >= 0.0)
@variable(leaf_11, z11[1:2])
@objective(leaf_11, Min, a*sum(z11[i]^2 for i in 1:2))
@constraint(leaf_11, ϵ*sum(x) + γ*y1 + δ*sum(z11) == s11)

leaf_12 = StructuredModel(parent=mid_model_1, id=12, prob=0.25)
# @variable(leaf_12, z12 >= 0.0)
@variable(leaf_12, z12[1:2])
@objective(leaf_12, Min, a*sum(z12[i]^2 for i in 1:2))
@constraint(leaf_12, ϵ*sum(x) + γ*y1 + δ*sum(z12) == s12)

leaf_21 = StructuredModel(parent=mid_model_2, id=21, prob=0.75)
# @variable(leaf_21, z21 >= 0.0)
@variable(leaf_21, z21[1:2])
@objective(leaf_21, Min, a*sum(z21[i]^2 for i in 1:2))
@constraint(leaf_21, ϵ*sum(x) + γ*y2 + δ*sum(z21) == s21)

leaf_22 = StructuredModel(parent=mid_model_2, id=22, prob=0.25)
# @variable(leaf_22, z22 >= 0.0)
@variable(leaf_22, z22[1:2])
@objective(leaf_22, Min, a*sum(z22[i]^2 for i in 1:2))
@constraint(leaf_22, ϵ*sum(x) + γ*y2 + δ*sum(z22) == s22)
;

# Progressive Hedging Solution

In [4]:
@time (n, err, obj, soln, phd) = PH.solve(root_model, 
                                          with_optimizer(Ipopt.Optimizer, print_level=0, tol=1e-12), 
                                          25.0, atol=1e-8, max_iter=500, report=false)
println("Number of iterations: ", n)
println("L^2 error: ", err)
println(obj)

      From worker 2:	
      From worker 2:	******************************************************************************
      From worker 2:	This program contains Ipopt, a library for large-scale nonlinear optimization.
      From worker 2:	 Ipopt is released as open source code under the Eclipse Public License (EPL).
      From worker 2:	         For more information visit http://projects.coin-or.org/Ipopt
      From worker 2:	******************************************************************************
      From worker 2:	
      From worker 3:	
      From worker 3:	******************************************************************************
      From worker 3:	This program contains Ipopt, a library for large-scale nonlinear optimization.
      From worker 3:	 Ipopt is released as open source code under the Eclipse Public License (EPL).
      From worker 3:	         For more information visit http://projects.coin-or.org/Ipopt
      From worker 3:	*******************************

In [5]:
soln

,variable,value,stage,scenarios
,String,Float64,Int64,String
1,x[1],7.5625,1,"0, 1, 2, 3"
2,x[2],2.52227e-9,1,"0, 1, 2, 3"
3,x[3],1.0,1,"0, 1, 2, 3"
4,y1,1.75,2,"0, 1"
5,y2,5.86835e-9,2,"2, 3"
6,z11[1],-0.65625,3,0
7,z11[2],-0.65625,3,0
8,z12[1],2.84375,3,1
9,z12[2],2.84375,3,1


In [6]:
# PH.retrieve_no_hats(phd)

In [7]:
# PH.retrieve_w(phd)

# Extensive Form Solution

In [8]:
@time ef_model = PH.solve_extensive_form(root_model, with_optimizer(Ipopt.Optimizer, print_level=0))
for var in JuMP.all_variables(ef_model)
    println(var, " = ", JuMP.value(var))
end
JuMP.objective_value(ef_model)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

 14.996311 seconds (45.08 M allocations: 2.216 GiB, 8.08% gc time)
x[2] = 0.0
x[3] = 1.000000007465833
x[1] = 7.562500011769159
y2 = 0.0
y1 = 1.7499999908010653
z21[2] = -1.7812499999655194
z21[1] = -1.7812499999655194
z22[2] = 4.71875000003448
z22[1] = 4.71875000003448
z11[2] = -0.656250000157248
z11[1] = -0.656250000157248
z12[2] = 2.843749999842752
z12[1] = 2.843749999842752


178.3537498476204

In [9]:
ef_model

A JuMP Model
Minimization problem with:
Variables: 13
Objective function type: GenericQuadExpr{Float64,VariableRef}
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 5 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 4 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: SolverName() attribute not implemented by the optimizer.